In [193]:
import pandas as pd

# Create a sample DataFrame
data = {
    'layout': ['USA1', 'USA1', 'USA1', 'USA2', 'USA2', 'USA2', 'USA3', 'USA3', 'USA3'],
    'scenario': [1, 2, 3, 1, 2, 3, 1, 2, 3],
    'historical': [True, True, False, True, False, False, True, False, False],
    'fast': [True, False, True, True, False, False, False, False, True],
    'big': [True, True, False, False, True, False, False, True, True],
}

df = pd.DataFrame(data)

import pandas as pd
import numpy as np
import random

# Define parameters
layouts = ['USA1', 'USA2', 'USA3']
scenarios = [1, 2, 3]
ground_sensors = ['strategy1', 'strategy2']
drones = ['dstrategy1', 'dstrategy2', 'dstrategy3']

# Generate all combinations
data = []
for layout in layouts:
    for scenario in scenarios:
        for gs in ground_sensors:
            for dr in drones:
                # Simulate detection time: 20% chance of being -1 (not detected)
                dt = round(random.uniform(0, 60), 1) if random.random() > 0.2 else -1
                data.append((layout, scenario, gs, dr, dt))

# Create DataFrame
results = pd.DataFrame(data, columns=['layout', 'scenario', 'ground_sensor', 'drone', 'detection_time'])

In [194]:
merged_df = df.merge(results, how='left', on=["layout", "scenario"])

# Subgroup Score

In [208]:

# Make sure 'detection_time' is in your columns
assert 'detection_time' in merged_df.columns, "'detection_time' not found in DataFrame columns"

# Step 1: Filter out -1 values for mean and std calculations
filtered_df = merged_df[merged_df['detection_time'] != -1]

# Step 2: Group and aggregate only valid values
agg_df = filtered_df.groupby(['ground_sensor', 'drone', 'fast', 'big']).agg(
    detection_time_mean=('detection_time', 'mean'),
    detection_time_std=('detection_time', 'std')
)

# Step 3: Format into a single string column
agg_df['detection_summary'] = agg_df.apply(
    lambda row: f"{row['detection_time_mean']:.2f} ({row['detection_time_std']:.2f})" 
    if pd.notnull(row['detection_time_mean']) and pd.notnull(row['detection_time_std']) 
    else "", axis=1
)

agg_df = agg_df.drop(columns=['detection_time_mean', 'detection_time_std'])


# Step 3: Define valid percentage calculation (based on original, unfiltered df)
def valid_percentage(subgroup):
    total = len(subgroup)
    valid = (subgroup != -1).sum()
    return (valid / total) * 100 if total > 0 else 0

# Apply percentage function on the unfiltered data
valid_pct = merged_df.groupby(['ground_sensor', 'drone', 'fast', 'big'])['detection_time'].apply(valid_percentage)
valid_pct.name = 'detection_time_valid_pct'

# Step 4: Combine results
result = agg_df.join(valid_pct, how='outer').reset_index()

result['summary'] = result.apply(
    lambda row: f"{row['detection_time_valid_pct']:.0f}%\n{row['detection_summary']}" 
    if pd.notnull(row['detection_time_valid_pct']) and pd.notnull(row['detection_summary']) 
    else "", axis=1
)

result = result.drop(columns=['detection_summary', 'detection_time_valid_pct'])

In [209]:
# Assuming 'result' is the DataFrame from previous steps:
pivoted_result = result.pivot_table(
    index=['ground_sensor', 'drone'],
    columns=['fast', 'big'],
    values=['summary'],
    aggfunc='first'
)

# Optional: flatten MultiIndex columns for easier viewing
pivoted_result.columns = ['_'.join(map(str, col)) for col in pivoted_result.columns]
pivoted_result = pivoted_result.reset_index()

In [210]:
results_pt2 = pivoted_result

# Overall Score

In [230]:
results_pt1 = results.groupby(['ground_sensor', 'drone']).agg(
    detection_time_mean=('detection_time', 'mean'),
    detection_time_std=('detection_time', 'std')
)

results_pt1['detection_summary'] = results_pt1.apply(
    lambda row: f"{row['detection_time_mean']:.2f} ({row['detection_time_std']:.2f})" 
    if pd.notnull(row['detection_time_mean']) and pd.notnull(row['detection_time_std']) 
    else "", axis=1
)

results_pt1 = results_pt1.drop(columns=['detection_time_mean', 'detection_time_std'])

# Apply percentage function on the unfiltered data
valid_pct = results.groupby(['ground_sensor', 'drone'])['detection_time'].apply(valid_percentage)
valid_pct.name = 'detection_time_valid_pct'

# Step 4: Combine results
result = results_pt1.join(valid_pct, how='outer').reset_index()

result['summary'] = result.apply(
    lambda row: f"{row['detection_time_valid_pct']:.0f}%\n{row['detection_summary']}" 
    if pd.notnull(row['detection_time_valid_pct']) and pd.notnull(row['detection_summary']) 
    else "", axis=1
)

result = result.drop(columns=['detection_summary', 'detection_time_valid_pct'])

result


,ground_sensor,drone,summary
0,strategy1,dstrategy1,78%\n34.18 (22.21)
1,strategy1,dstrategy2,67%\n17.53 (16.91)
2,strategy1,dstrategy3,67%\n13.84 (15.18)
3,strategy2,dstrategy1,100%\n34.04 (16.97)
4,strategy2,dstrategy2,78%\n20.16 (17.57)
5,strategy2,dstrategy3,89%\n30.23 (24.49)


In [231]:
results_pt1 = result
results_pt1

,ground_sensor,drone,summary
0,strategy1,dstrategy1,78%\n34.18 (22.21)
1,strategy1,dstrategy2,67%\n17.53 (16.91)
2,strategy1,dstrategy3,67%\n13.84 (15.18)
3,strategy2,dstrategy1,100%\n34.04 (16.97)
4,strategy2,dstrategy2,78%\n20.16 (17.57)
5,strategy2,dstrategy3,89%\n30.23 (24.49)


# Historical Score

In [232]:
hist_df = merged_df[merged_df['historical'] == True]

results_pt3 = hist_df.groupby(['ground_sensor', 'drone']).agg(
    detection_time_mean=('detection_time', 'mean'),
    detection_time_std=('detection_time', 'std')
)

results_pt3['detection_summary'] = results_pt3.apply(
    lambda row: f"{row['detection_time_mean']:.2f} ({row['detection_time_std']:.2f})" 
    if pd.notnull(row['detection_time_mean']) and pd.notnull(row['detection_time_std']) 
    else "", axis=1
)
results_pt3 = results_pt3.drop(columns=['detection_time_mean', 'detection_time_std'])


# Apply percentage function on the unfiltered data
valid_pct = hist_df.groupby(['ground_sensor', 'drone'])['detection_time'].apply(valid_percentage)
valid_pct.name = 'detection_time_valid_pct'

# Step 4: Combine results
result = results_pt3.join(valid_pct, how='outer').reset_index()

result['summary'] = result.apply(
    lambda row: f"{row['detection_time_valid_pct']:.0f}%\n{row['detection_summary']}" 
    if pd.notnull(row['detection_time_valid_pct']) and pd.notnull(row['detection_summary']) 
    else "", axis=1
)

result = result.drop(columns=['detection_summary', 'detection_time_valid_pct'])

results_pt3 = result
results_pt3

,ground_sensor,drone,summary
0,strategy1,dstrategy1,75%\n34.10 (24.90)
1,strategy1,dstrategy2,75%\n16.77 (12.57)
2,strategy1,dstrategy3,100%\n23.57 (15.77)
3,strategy2,dstrategy1,100%\n20.92 (12.55)
4,strategy2,dstrategy2,100%\n35.33 (7.23)
5,strategy2,dstrategy3,100%\n42.38 (18.34)


In [233]:
results_merged = results_pt1.merge(
    results_pt3, how='left', on=['ground_sensor', 'drone'], suffixes=('', '_historical')).merge(
        results_pt2, how='left', on=['ground_sensor', 'drone'])

In [234]:
results_merged

,ground_sensor,drone,summary,summary_historical,summary_False_False,summary_False_True,summary_True_False,summary_True_True
0,strategy1,dstrategy1,78%\n34.18 (22.21),75%\n34.10 (24.90),100%\n46.00 (13.01),67%\n53.45 (8.27),100%\n41.10 (9.19),50%\n
1,strategy1,dstrategy2,67%\n17.53 (16.91),75%\n16.77 (12.57),100%\n35.35 (21.57),67%\n19.35 (0.07),,100%\n25.70 (4.10)
2,strategy1,dstrategy3,67%\n13.84 (15.18),100%\n23.57 (15.77),50%\n,67%\n13.70 (2.40),50%\n,100%\n24.30 (4.24)
3,strategy2,dstrategy1,100%\n34.04 (16.97),100%\n20.92 (12.55),100%\n38.70 (28.71),100%\n31.63 (24.84),100%\n34.85 (2.19),100%\n32.20 (13.15)
4,strategy2,dstrategy2,78%\n20.16 (17.57),100%\n35.33 (7.23),100%\n24.45 (24.25),67%\n23.15 (25.95),100%\n30.05 (0.07),50%\n
5,strategy2,dstrategy3,89%\n30.23 (24.49),100%\n42.38 (18.34),100%\n20.05 (26.52),100%\n28.30 (27.42),50%\n,100%\n46.50 (15.84)


In [236]:
latex_str = results_merged.to_latex(
    index=False,
    escape=False,
    formatters={"name": str.upper},
    float_format="{:.3f}".format,
)

# Optional: escape percent signs
latex_str = latex_str.replace('%', r'\%')

# Insert \midrule after every data row (i.e., every '\\' that ends a row)
lines = latex_str.splitlines()
new_lines = []

for line in lines:
    new_lines.append(line)
    if line.strip().endswith('\\\\') and not line.strip().startswith(r'\toprule') and not line.strip().startswith(r'\midrule') and not line.strip().startswith(r'\bottomrule'):
        new_lines.append(r'\midrule')

# Join back into one LaTeX string
final_latex = '\n'.join(new_lines)

with open("leaderboards.txt", "w") as f:
    f.write(final_latex)